In [2]:
import numpy as np
import pandas as pd

In [49]:
a, b = 1.5, 2.0
# a, b = 0.6, 1.1

i = 10
h = (b - a) / i

x_points = np.array([1.52, 1.52, 1.97, 1.77])
# x_points = np.array([0.88])

def y(x: float) -> float:
    """
    Function to compute the given mathematical expression.

    Args:
    x (float): Input value

    Returns:
    float: Computed result
    """
    return np.pow(x, 2) + np.log(x) - 4

# def y(x: float) -> float:
#     return x - np.sin(np.pi * x)

x_val = np.arange(a, a + h * (i + 1), h)
y_val = y(x_val)
x_val, y_val

(array([1.5 , 1.55, 1.6 , 1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  ]),
 array([-1.34453489, -1.15924507, -0.96999637, -0.77672471, -0.57937175,
        -0.37788421, -0.17221334,  0.03768564,  0.25185389,  0.47032937,
         0.69314718]))

### Оценим ощибку интерполяции первого порядка при помощи производной

$$R_1(x) = \frac{f''(ξ) · ω_2 (x)}{2},$$ где $$ξ ∈ [x_i, x_i+1],$$ а $$ω_2(x) = (x−x_i)·(x−x_i+1)$$

Функция: $$f(x)=x^2+ ln(x) − 4$$

2-ая производная функции:  $$f''(x) = 2 - \frac{1}{x^2}$$

На данном промежутке (на ближайших к точке 1.52 узлах) $[1.5, 1.55]$ она принимает минимальное значение в точке $1.5$ и максимальное в точке $1.55$

$$f''_{\max} = 1.58, f′′_{\min} = 1.56$$

$$ω_2(x^*) = (x^* - 1.5)·(x^* - 1.55)$$

$$ω_2(1.52) = -0.0006$$



$$|R_2|_{\max}(1.52) = \left|\frac{ω_2(x^*) · f''_{\max}}{2} \right| = \left|\frac{-0.0006 · 1.58}{2} \right| = 0.000475$$
$$|R_1|_{\min}(1.52)  = \left|\frac{ω_2(x^*) · f''_{\min}}{2} \right| = \left|\frac{-0.0006 · 1.56}{2}\right| = 0.000468$$


In [39]:
def lagrange_polynomial(x: float, x_values: np.ndarray, k: int) -> float:
    """
    Computes Lagrange basis polynomial.

    Args:
    x (float): The x-value
    x_values (np.ndarray): Known x-values
    k (int): Index
    
    Returns:
    float: Basis polynomial value
    """
    x_k = x_values[k]
    res = np.prod([( (x - x_values[i]) / (x_k - x_values[i]) ) for i in range(len(x_values)) if i != k])
    return res

def lagrange_interpolation(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs Lagrange interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    res = np.sum([( y_values[i] * lagrange_polynomial(x, x_values, i) ) for i in range(len(x_values))])
    return res

def lagrange_interpolation_error(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Computes error of Lagrange interpolation.

    Args:
    x (float): The x-value to check error
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Error value
    """
    return lagrange_interpolation(x, x_values, y_values) - y(x)

In [40]:
def linear_interpolation_lagrange(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs linear interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    i = np.searchsorted(x_values, x)
    
    res = lagrange_interpolation(x, x_values[i:i+2], y_values[i:i+2])
    return res

def linear_interpolation_lagrange_error(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Computes the error of linear interpolation.

    Args:
    x (float): The x-value to check error
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolation error
    """
    return linear_interpolation_lagrange(x, x_values, y_values) - y(x)

### Оценка ошибки интерполяции второго порядка

Используем формулу остаточного члена интерполяционной формулы Лагранжа второго порядка:

$$R_2(x) = \frac{f'''(\xi) \cdot \omega_3(x)}{6}, \quad \xi \in [x_{i-1}, x_{i+1}]$$

где  

$$
\omega_3(x) = (x - x_{i-1})(x - x_i)(x - x_{i+1})
$$

#### 1. Вычислим третью производную функции

Функция:

$$
f(x) = x^2 + \ln(x) - 4
$$
Третья производная:

$$
f'''(x) = \frac{2}{x^3}
$$

$$
f'''(1.5) = f'''_{\max} \approx 0.5926
$$

$$
f'''(1.6) = f'''_{\min} = 0.4882
$$

Узлы интерполяции, ближайшие к точке 1.52:  
$x_{i-1} = 1.5$, $x_i = 1.55$, $x_{i+1} = 1.6$

$$
\omega_3(x) = (x - 1.5)(x - 1.55)(x - 1.6)
$$

$$
\omega_3(1.52) = 0.000048
$$

$$
|R_2|_{\max}(1.52) = \left| \frac{f'''_{\max} \cdot \omega_3(1.52)}{6} \right| = \left| \frac{0.5926 \cdot 0.000048}{6} \right| \approx 1.42e-5
$$

$$
|R_2|_{\min}(1.52) = \left| \frac{f'''_{\max} \cdot \omega_3(1.52)}{6} \right| = \left| \frac{0.4882 \cdot 0.000048}{6} \right| \approx 1.17e-5
$$



In [41]:
def square_interpolation_lagrange(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs square interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    i = np.searchsorted(x_values, x) 
    
    if i == 0:
        i = 1
    elif i >= len(x_values) - 1:
        i = len(x_values) - 2
    
    res = lagrange_interpolation(x, x_values[i-1:i+2], y_values[i-1:i+2])
    return res

def square_interpolation_lagrange_error(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Computes the error of square interpolation.

    Args:
    x (float): The x-value to check error
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolation error
    """
    return square_interpolation_lagrange(x, x_values, y_values) - y(x)

In [42]:
def newton_interpolation(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs Newton interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    n = len(x_values)
    
    divided_differences = np.zeros((n, n))
    
    for i in range(n):
        divided_differences[i][0] = y_values[i]
        
    for j in range(1,n):
        for i in range(n-j):
            divided_differences[i][j] = (divided_differences[i + 1][j - 1] - divided_differences[i][j - 1]) / (x_values[i + j] - x_values[i])
            
    res = y_values[0] + np.sum([divided_differences[0][i] * np.prod(x - x_values[:i]) for i in range(1, n)])
    return res

def newton_interpolation_error(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Computes error of Newton interpolation.

    Args:
    x (float): The x-value to check error
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Error value
    """
    return newton_interpolation(x, x_values, y_values) - y(x)

In [50]:
data = []
for point in x_points:
    data.append([point, 
                 y(point), 
                 linear_interpolation_lagrange(point, x_val, y_val), 
                 linear_interpolation_lagrange_error(point, x_val, y_val),
                 square_interpolation_lagrange(point, x_val, y_val), 
                 square_interpolation_lagrange_error(point, x_val, y_val), 
                 lagrange_interpolation(point, x_val, y_val), 
                 lagrange_interpolation_error(point, x_val, y_val), 
                 newton_interpolation(point, x_val, y_val), 
                 newton_interpolation_error(point, x_val, y_val)])

df = pd.DataFrame(data, columns=["Точка", "Истинное значение", "Линейная Лагранж", "Ошибка линейная Лагранж", "Квадратичная Лагранж", "Ошибка квадратичная Лагранж", "Лагранж 11 узлов", "Ошибка Лагранжа 11 узлов", "Ньютон", "Ошибка Ньютона"])
df

,Точка,Истинное значение,Линейная Лагранж,Ошибка линейная Лагранж,Квадратичная Лагранж,Ошибка квадратичная Лагранж,Лагранж 11 узлов,Ошибка Лагранжа 11 узлов,Ньютон,Ошибка Ньютона
0,1.52,-1.270890,-1.272794,-0.001905,-1.270894,-0.000004,-1.270890,-4.238832e-13,-1.270890,-4.249934e-13
1,1.52,-1.270890,-1.272794,-0.001905,-1.270894,-0.000004,-1.270890,-4.238832e-13,-1.270890,-4.249934e-13
2,1.97,0.558934,0.693147,0.134214,0.558935,0.000002,0.558934,2.128298e-13,0.558934,2.131628e-13
3,1.77,-0.296120,-0.298153,-0.002032,-0.296123,-0.000003,-0.296120,4.107825e-15,-0.296120,4.218847e-15


## *Полученные ошибки оказались чуть больше, определенных для значений ошибки, найденных теоретически для точки 1.52 для линейной интерполяции и чуть меньше для квадратичной интерполяции*